In [2]:
from nltk.corpus import cmudict

In [8]:
cmudict.entries()[100]

('abdication', ['AE2', 'B', 'D', 'IH0', 'K', 'EY1', 'SH', 'AH0', 'N'])

In [5]:
cmudict.dict()['hello']

[['HH', 'AH0', 'L', 'OW1'], ['HH', 'EH0', 'L', 'OW1']]

In [48]:
def MED(w1, w2):
    cost = []
    del_cost = 1
    add_cost = 1
    sub_cost = 2
    
    vowels = ['A', 'E', 'I', 'O', 'U']
    voiced = ['B', 'D', 'G', 'J', 'L', 'M', 'N', 'R', 'V', 'W', 'Y', 'Z']
    unvoiced = ['C', 'F', 'H', 'K', 'P', 'S', 'T']
    
    for i in range(len(w1)+1):
        x = []
        for j in range(len(w2)+1):
            x.append(0)
        cost.append(x)
    
    for i in range(len(w1)+1):
        cost[i][0] = i
    for j in range(len(w2)+1):
        cost[0][j] = j
        
    for i in range(1, len(w1)+1):
        for j in range(1, len(w2)+1):
            
            # if they are the same:
            if w1[i-1] == w2[j-1]:
                del_total = cost[i-1][j] + 10
                add_total = cost[i][j-1] + 10
                sub_total = cost[i-1][j-1]
            
            # if they are both vowels:
            elif w1[i-1][0] in vowels and w2[j-1][0] in vowels:
                del_total = cost[i-1][j] + del_cost
                add_total = cost[i][j-1] + add_cost
                sub_total = cost[i-1][j-1] + sub_cost
                    
            # if only one is a vowel:
            elif w1[i-1][0] in vowels or w2[j-1][0] in vowels:
                del_total = cost[i-1][j] + del_cost + 1
                add_total = cost[i][j-1] + add_cost + 1
                sub_total = cost[i-1][j-1] + sub_cost + 1
                    
            # if both are voiced consonants:
            elif w1[i-1][0] in voiced and w2[j-1][0] in voiced:
                del_total = cost[i-1][j] + del_cost
                add_total = cost[i][j-1] + add_cost
                sub_total = cost[i-1][j-1] + 1
            
            # if both are unvoiced consonants:
            elif w1[i-1][0] in voiced and w2[j-1][0] in unvoiced:
                del_total = cost[i-1][j] + del_cost
                add_total = cost[i][j-1] + add_cost
                sub_total = cost[i-1][j-1] + 1
            
            # if one is voiced and the other is unvoiced:
            else:
                del_total = cost[i-1][j] + del_cost
                add_total = cost[i][j-1] + add_cost
                sub_total = cost[i-1][j-1] + sub_cost
                    
            options = [del_total, add_total, sub_total]
            options.sort()
            cost[i][j] = options[0]
           
    return cost[-1][-1]

In [49]:
w = 'test'
w2 = 'beast'
res = MED(cmudict.dict()[w][0], cmudict.dict()[w2][0])

In [50]:
res

4

In [24]:
print(cmudict.dict()[w])

[['T', 'EH1', 'S', 'T']]


In [78]:
w = 'states'
word1 = cmudict.dict()[w][0]
options = []
for entry in cmudict.entries():
    price = MED(word1, entry[1])
    if price < 3 and w not in entry[0]:
        options.append(entry[0])

In [79]:
len(options)

47

In [80]:
options[:10]

['ates',
 'aytes',
 "eight's",
 'eights',
 'est',
 'estate',
 "estate's",
 'sate',
 'sates',
 'scates']